# Before Run the Code

[国会会議録検索システム検索用API](http://kokkai.ndl.go.jp/api.html)

# Make URL

In [1]:
import urllib.request
import urllib.parse #参考: http://d.hatena.ne.jp/motie/20100728/1280335730

def DietSearch(search):
    request = "http://kokkai.ndl.go.jp/api/1.0/speech?" + urllib.parse.quote_plus(search,encoding="utf-8")

# Get XML file

In [17]:
def searchKeyword(speaker="", startRec=1, any="", Dfrom="0000-01-01", Duntil="9999-12-31"):
    if len(any) > 0:
        search = "startRecord=" + str(startRec) + "&" + "any=" + any + "&" + "speaker=" + speaker + "&from=" + Dfrom + "&until=" + Duntil
    elif speaker== "":
        search = "startRecord=" + str(startRec) + "&from=" + Dfrom + "&until=" + Duntil
    else:
        search =  "startRecord=" + str(startRec) + "&" + "speaker=" + speaker + "&from=" + Dfrom + "&until=" + Duntil
    return search

In [20]:
import httplib2
import time
def DietSearch(search):
    request = "http://kokkai.ndl.go.jp/api/1.0/speech?" + urllib.parse.quote_plus(search,encoding="utf-8")

    h = httplib2.Http('.cache') #by httplib2
    response, content = h.request(request)
    #content = content.decode("utf-8") #etree.fromstringをするときに、"Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration."というエラーを回避するためにこの行は使わない
    time.sleep(1.2) # Be nice to servers!
    
    return content

# Extract Information

In [21]:
from lxml import etree

def XMLextractSpeech(XMLcontent):
    root = etree.fromstring(XMLcontent) # 文字列からの読み込み

    ##第二階層の情報を取得##
    numberOfRecords = root.find('numberOfRecords').text #リクエストされた検索条件に該当する総件数
    numberOfReturn = root.find('numberOfReturn').text #当該リクエストで返戻した件数
    startRecord = root.find('startRecord').text #当該リクエストの返戻結果に含まれる最初のレコードの総結果件数の中での出現位置
    try: #次開始位置はない場合もあるので
        nextRecordPosition = 0
        nextRecordPosition = root.find('nextRecordPosition').text #総結果件数が今回の返戻件数を超え、続きを取得したい場合に、次に指定すべき開始位置。次の開始レコードが存在しない場合は、タグも含め表示されない
    except AttributeError:
        nextRecordPosition = 0 #構文法的には文が必要だが、コードとしては何も実行したくない場合のプレースホルダとして用いる


    ##第三階層以降の情報(実際の発言など)を取得する## --> numberOfReturnと同じことみたい
    #numRecords = 0 #次のfor loopで調べるrecordの数
    #for child in root[4]:
    #    numRecords += 1

    m=4 #<nextRecordPosition>のあるなしで、どこから始めるかが変わるみたい
    if nextRecordPosition == 0:
        m=3

    extracted = []
    for i in range(int(numberOfReturn)):
        temp_extracted = []
        for neighbor in root[m][i].iter(): # records(第二階層5つ目の要素)に入っているrecordうち何番目かを２つ目の[]が指定している
            temp_extracted.append([neighbor.tag, neighbor.text])
        extracted.append(temp_extracted)

    return ([nextRecordPosition, extracted])

In [5]:
def recordDiet(speaker, startRec=1, any="", Dfrom="0000-01-01", Duntil="9999-12-31"):
    search = searchKeyword(speaker, startRec, any, Dfrom, Duntil)
    XMLcontent = DietSearch(search)
    temp = XMLextractSpeech(XMLcontent)
    newStart = int(temp[0])
    extracted = temp[1]

    while newStart != 0: #newStartが0じゃない、つまり先がある限り続ける
        search = searchKeyword(speaker, newStart, any, Dfrom, Duntil)
        XMLcontent = DietSearch(search)
        temp = XMLextractSpeech(XMLcontent)
        newStart = int(temp[0])
        extracted.extend(temp[1])

    return extracted

# Clean Text and Make a Table

In [6]:
# recordDietの結果をPandasにまとめる
import pandas as pd
import os.path

def cleanText(text):
    cleaned = text.replace('\n', ' ') # 改行記号を除く
    cleaned = cleaned.strip() # 半角スペースを除く
    cleaned = cleaned.replace("　", "") # 全角スペースを除く

    return cleaned

def intoDataFrame(extracted):
    #各発言ごとに、リスト形式のものをdictに直してからpandasに渡す

    count = 1
    for i in extracted:
        if count == 1:
            ext_df = pd.DataFrame([{cleanText(k): cleanText(v) for [k, v] in i}]) # If each dict represents a row, you could pass a list of dicts (http://goo.gl/t0uCHp)
        else:
            ext_df = ext_df.append(pd.DataFrame([{cleanText(k): cleanText(v) for [k, v] in i}])) # dataframeの結合絡み (http://goo.gl/8H6ER8)
        count += 1

    # indexの設定
    ext_df.index = range(0, len(ext_df)) # cf. http://inaz2.hatenablog.com/entry/2013/04/09/002318

    return ext_df

# Save

In [7]:
def resultsSave(ext_df, path="/Users/", name="diet",type="csv"):
# 2種類の形式での保存を可能にする
    if type=="csv":
        save_path = os.path.join(path, (name + ".csv"))
        ext_df.to_csv(save_path) # UTF-8で保存されている
    elif type=="pandas":
        save_path = os.path.join(path, (name + ".pkl"))
        ext_df.to_pickle(save_path)
    else:
        print("File type you specified is not defined")

# Let's Use!

In [11]:
ext_df = intoDataFrame(recordDiet(speaker="安倍晋三", any="アベノミクス", Dfrom="2015-03-01", Duntil="2016-05-20"))

In [13]:
ext_df[:3]

,date,issue,meetingURL,nameOfHouse,nameOfMeeting,pdfURL,record,recordData,session,speaker,speech,speechOrder,speechRecord
0,2016-05-18,1号,http://kokkai.ndl.go.jp/SENTAKU/ryoin/190/9001...,両院,国家基本政策委員会合同審査会,http://kokkai.ndl.go.jp/SENTAKU/ryoin/190/9001...,,,190,安倍晋三,○内閣総理大臣（安倍晋三君）本日、速報値が出ました。年率でいえば名目二％、そして実質一・七％...,5,
1,2016-05-16,20号,http://kokkai.ndl.go.jp/SENTAKU/syugiin/190/00...,衆議院,予算委員会,http://kokkai.ndl.go.jp/SENTAKU/syugiin/190/00...,,,190,安倍晋三,○安倍内閣総理大臣三年半が経過いたしました。 我々は、政権を奪還する際に、国民総所得、ＧＮＩ...,41,
2,2016-03-29,20号,http://kokkai.ndl.go.jp/SENTAKU/sangiin/190/00...,参議院,予算委員会,http://kokkai.ndl.go.jp/SENTAKU/sangiin/190/00...,,,190,安倍晋三,○内閣総理大臣（安倍晋三君）現在御審議をいただいている平成二十八年度予算は、一億総活躍社会の...,102,


In [15]:
resultsSave(ext_df, path="/Users/", name="diet",type="csv")

In [22]:
ext_df2 = intoDataFrame(recordDiet(speaker="", 
                                   any="アベノミクス", Dfrom="2015-03-01", Duntil="2015-03-10"))

In [24]:
ext_df2[:1]

,date,issue,meetingURL,nameOfHouse,nameOfMeeting,pdfURL,record,recordData,session,speaker,speech,speechOrder,speechRecord
0,2015-03-10,1号,http://kokkai.ndl.go.jp/SENTAKU/syugiin/189/00...,衆議院,予算委員会第六分科会,http://kokkai.ndl.go.jp/SENTAKU/syugiin/189/00...,,,189,鈴木義弘,○鈴木（義）分科員ありがとうございます。 これも過去に質問したときに事例で使わせてもらったん...,355,


In [25]:
resultsSave(ext_df2, path="/Users/", name="diet2",type="csv")